In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from pyBedGraph import BedGraph
from pybedtools import BedTool
import scipy.stats
from collections import Counter

In [46]:
def read_peakfile(directory, file_name):
    with open(directory + file_name) as f:
        crnpk = {}
        #next(f)
        for line in f:
            tmp = line.strip().split("\t")[:-1]
            region = tmp[0]+","+tmp[1]+","+tmp[2] +','+ tmp[3]+','+tmp[4]+','+tmp[5] +";" + tmp[9]
            tmp[1] = int(tmp[1]) # peak start
            tmp[2] = int(tmp[2]) # peak end
            tmp[7] = int(tmp[7]) # gene start
            tmp[8] = int(tmp[8]) # gene end
            tmp[13] = float(tmp[13]) # TPM
            tmp[14] = float(tmp[14]) # RPKM
            strand = tmp[9]
            if strand == '+': # positive strand
                if tmp[7] > tmp[1] and tmp[7] < tmp[2]:
                    dist = 0
                else:
                    dist = min(abs(tmp[7]-tmp[2]), abs(tmp[7]-tmp[1]))
            else: # negative strand
                if tmp[8] > tmp[1] and tmp[8] < tmp[2]:
                    dist = 0
                else:
                    dist = min(abs(tmp[8]-tmp[2]), abs(tmp[8]-tmp[1]))
            tmp.append(dist) #distance to promoter;  tmp[15]
            tmp.append(tmp[8]-tmp[7]) # gene length; tmp[16]
            #if 'UBE4B' in tmp and tmp[15] < 5000 and tmp[13] > 0.5 and tmp[16] > 5000:
                #print(tmp)
            if tmp[15] < 5000 and tmp[13] > 0.5 and tmp[16]>3000: # distance to promoter < 5kb and TMP > 0.5 and gene length > 3000
                if region not in crnpk.keys():
                    crnpk[region] = [tmp]
                else:
                    crnpk[region].append(tmp)
            else:
                tmp = [tmp[0], tmp[1], tmp[2], tmp[3], tmp[4], tmp[5],'.', 0, 0, '.', '.', '.', '.', 0, 0, 0, 0]
                if region not in crnpk.keys():
                    crnpk[region] = [tmp]
                else:
                    if tmp not in crnpk[region]:
                        crnpk[region].append(tmp)
    return crnpk

In [47]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [48]:
directory = '/Users/kimm/Desktop/GM12878_files/'
peak_file = 'RNAPII-ChIA-PET-drop_peaks_merge500bp-superenhancer_const_chromHMM_all_annot_ENCFF879KFK_20200728.bed'

In [49]:
peak = read_peakfile(directory, peak_file)

In [50]:
len(Counter([x.split(";")[0] for x in peak.keys()]).keys())

34107

In [51]:
filtered = {}
for key, val in peak.items():
    final = val[0]
    for x in val:
        if x[13] > final[13]: ## tpm
            if x[11] == final[11]: ## gene id same
                if x[16] > final[16]: # max length of gene 
                    final = x
                else: 
                    final[13] = x[13]
            else: 
                final = x
    if key ==','.join(['chr1', '10032500', '10035000', '.', '.', 'RAP40'])+';'+'+':
        print("final is: ")
        print(final)
    if key.split(';')[0] not in filtered.keys():
        filtered[key.split(';')[0]] = [[],[]]
    if final[9] == '+':
        filtered[key.split(';')[0]][0] = final
    elif final[9] == '-':
        filtered[key.split(';')[0]][1] = final

final is: 
['chr1', 10032500, 10035000, '.', '.', 'RAP40', 'chr1', 10032958, 10181239, '+', 'ENST00000343090', 'ENSG00000130939', 'UBE4B', 2.59, 1.99, 0, 148281]


In [53]:
cnts = [str(len(x[0]))+','+str(len(x[1])) for x in filtered.values()]

In [54]:
Counter(cnts)

Counter({'0,0': 19989, '17,17': 1903, '17,0': 6181, '0,17': 6034})

In [61]:
sum(Counter(cnts).values())

34107

In [65]:
Counter([key.split(",")[-1][0:3] for key, val in filtered.items() if len(val[0])==0 and len(val[1])==0])

Counter({'ROT': 6080, 'REN': 11867, 'RWP': 929, 'RPE': 296, 'RAP': 817})

In [66]:
towrite = []
for key, val in filtered.items():
    bed = key.split(',')
    #print(bed)
    if len(val[0]) == 0 :
        if len(val[1]) == 0: # no gene transcription
            bed.extend(['.', '.', '.','.','.','.','.','.'])
        else: # reverse transcription
            bed.extend(val[1][6:14])
    else:
        if len(val[1]) == 0: # forward transcription
            bed.extend(val[0][6:14])
        else: # both transcription
            if val[0][13] > val[1][13]: # TPM of forward > TPM of reverse
                bed.extend(val[0][6:14]) # forward
            else:
                bed.extend(val[1][6:14]) # reverse
    towrite.append(bed)
    del bed

In [71]:
write_result(directory, towrite, 'RNAPII-ChIA-PET-drop_peaks_merge500bp-superenhancer_const_chromHMM_ENCFF879KFK_20200729.bed')